### Load & clean the dataset

In [1]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-04-08 09:47:34--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.186.207, 142.251.116.207, 142.250.138.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.186.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  61.8MB/s    in 0.2s    

2025-04-08 09:47:34 (61.8 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2025-04-08 09:47:34--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.115.207, 142.250.138.207, 142.250.113.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.115.207|:443... connected.
HTTP request sent, awaiti

In [2]:
import pandas as pd

df = pd.read_csv('data/full_dataset/goemotions_1.csv')
#df2 = pd.read_csv('data/full_dataset/goemotions_2.csv')
#df3 = pd.read_csv('data/full_dataset/goemotions_3.csv')

#df = pd.concat([df1, df2, df3], ignore_index=True)
df.head(2)

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  70000 non-null  object 
 1   id                    70000 non-null  object 
 2   author                70000 non-null  object 
 3   subreddit             70000 non-null  object 
 4   link_id               70000 non-null  object 
 5   parent_id             70000 non-null  object 
 6   created_utc           70000 non-null  float64
 7   rater_id              70000 non-null  int64  
 8   example_very_unclear  70000 non-null  bool   
 9   admiration            70000 non-null  int64  
 10  amusement             70000 non-null  int64  
 11  anger                 70000 non-null  int64  
 12  annoyance             70000 non-null  int64  
 13  approval              70000 non-null  int64  
 14  caring                70000 non-null  int64  
 15  confusion          

In [4]:
# Identify emotion columns starting from "admiration"
cols = df.columns.tolist()
emotion_start_idx = cols.index("admiration")
emotion_cols = cols[emotion_start_idx:]

# Create a new column with only the first emotion
def get_first_emotion(row):
    for emotion in emotion_cols:
        if row[emotion] == 1:
            return emotion
    return None  # In case there's no emotion tagged

df['label'] = df.apply(get_first_emotion, axis=1)

# Preview
print(df[['text', 'label']].head())

                                                text    label
0                                    That game hurt.  sadness
1   >sexuality shouldn’t be a grouping category I...     None
2     You do right, if you don't care then fuck 'em!  neutral
3                                 Man I love reddit.     love
4  [NAME] was nowhere near them, he was by the Fa...  neutral


In [5]:
from sklearn.preprocessing import LabelEncoder

df['label'] = df.apply(get_first_emotion, axis=1)
df = df[df['label'].notna()].reset_index(drop=True)

# Encode string labels into integers
label_encoder = LabelEncoder()
df['label_id'] = label_encoder.fit_transform(df['label'])

### Tokenizing & data loader

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import ast

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

texts = df['text'].astype(str).tolist()
labels = df['label_id'].tolist()

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.1, random_state=42, stratify=labels
)

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
val_dataset = EmotionDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/jupyterhub/pyvenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Model architect:

In [8]:
!pip install huggingface_hub[hf_xet]
from transformers import BertForSequenceClassification

num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training

In [9]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
from tqdm import tqdm

EPOCHS = 5
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Average training loss: {total_loss / len(train_loader):.4f}")

Epoch 1/5


100%|██████████| 3874/3874 [05:27<00:00, 11.83it/s]


Average training loss: 2.1172
Epoch 2/5


100%|██████████| 3874/3874 [05:27<00:00, 11.84it/s]


Average training loss: 1.8233
Epoch 3/5


...

Average training loss: 1.6369
Epoch 4/5


100%|██████████| 3874/3874 [05:27<00:00, 11.84it/s]


Average training loss: 1.4531
Epoch 5/5


100%|██████████| 3874/3874 [05:28<00:00, 11.80it/s]

Average training loss: 1.2813


### Evaluation

In [10]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_true.extend(labels.cpu().numpy())

print(classification_report(all_true, all_preds, target_names=label_encoder.classes_))

                precision    recall  f1-score   support

    admiration       0.45      0.64      0.53       565
     amusement       0.48      0.64      0.55       293
         anger       0.37      0.33      0.35       255
     annoyance       0.21      0.22      0.22       391
      approval       0.26      0.22      0.24       521
        caring       0.26      0.33      0.29       170
     confusion       0.32      0.39      0.35       223
     curiosity       0.34      0.41      0.37       260
        desire       0.30      0.34      0.32        98
disappointment       0.17      0.16      0.16       222
   disapproval       0.28      0.24      0.26       296
       disgust       0.24      0.25      0.24       109
 embarrassment       0.26      0.20      0.22        56
    excitement       0.20      0.18      0.19       147
          fear       0.28      0.37      0.32        82
     gratitude       0.64      0.68      0.66       282
         grief       0.50      0.24      0.32  

In [11]:
model.save_pretrained("./bert_text_model")
tokenizer.save_pretrained("./bert_text_model")

('./bert_text_model/tokenizer_config.json',
 './bert_text_model/special_tokens_map.json',
 './bert_text_model/vocab.txt',
 './bert_text_model/added_tokens.json')